In [ ]:
#1&2
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def get_amazon_products(search_query):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://www.amazon.in/')
    
    # Enter the search query
    search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
    search_box.send_keys(search_query)
    search_box.submit()
    
    product_details = []
    current_page = 1
    max_pages = 3

    while current_page <= max_pages:
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for product in products:
            try:
                title = product.h2.text.strip()
                url = 'https://www.amazon.in' + product.h2.a['href']
                price = product.find('span', 'a-price-whole').text.strip()
                try:
                    brand = product.find('span', 'a-size-base-plus a-color-base a-text-normal').text.strip()
                except:
                    brand = "-"
                try:
                    availability = product.find('span', 'a-size-base a-color-price').text.strip()
                except:
                    availability = "-"
                
                product_details.append({
                    'Brand Name': brand,
                    'Name of the Product': title,
                    'Price': price,
                    'Return/Exchange': '-',  # Placeholder as it's not available on the search results page
                    'Expected Delivery': '-',  # Placeholder as it's not available on the search results page
                    'Availability': availability,
                    'Product URL': url
                })
            except:
                continue
        
        current_page += 1
        try:
            next_page = driver.find_element(By.CLASS_NAME, 's-pagination-next')
            next_page.click()
        except:
            break
    
    driver.quit()
    
    df = pd.DataFrame(product_details)
    df.to_csv(f'{search_query}_amazon_products.csv', index=False)
    return df

# Input from user
search_query = input("Enter the product to search on Amazon: ")
df = get_amazon_products(search_query)
print(df)


In [ ]:
#3
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os

def download_google_images(keywords, num_images):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://images.google.com/')
    
    for keyword in keywords:
        search_box = driver.find_element(By.NAME, 'q')
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.submit()
        
        time.sleep(2)
        images = driver.find_elements(By.CSS_SELECTOR, 'img.rg_i.Q4LuWd')
        
        if not os.path.exists(keyword):
            os.makedirs(keyword)
        
        for i in range(min(num_images, len(images))):
            try:
                images[i].click()
                time.sleep(2)
                img_url = driver.find_element(By.CSS_SELECTOR, 'img.n3VNCb').get_attribute('src')
                img_data = requests.get(img_url).content
                with open(f'{keyword}/{keyword}_{i+1}.jpg', 'wb') as handler:
                    handler.write(img_data)
            except:
                continue
    
    driver.quit()

keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
download_google_images(keywords, 10)


In [ ]:
#4
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def get_flipkart_smartphones(search_query):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://www.flipkart.com/')
    
    # Close the login popup
    try:
        close_login_popup = driver.find_element(By.XPATH, '//button[contains(text(),"✕")]')
        close_login_popup.click()
    except:
        pass
    
    # Enter the search query
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.submit()
    
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    smartphones = soup.find_all('div', '_1AtVbE')
    
    smartphone_details = []

    for phone in smartphones:
        try:
            title = phone.find('a', 'IRpwTa').text.strip()
            url = 'https://www.flipkart.com' + phone.find('a', '_1fQZEK')['href']
            price = phone.find('div', '_30jeq3 _1_WHN1').text.strip()
            
            details = phone.find_all('li', 'rgWa7D')
            specs = {d.text.split(":")[0].strip(): d.text.split(":")[1].strip() for d in details}
            
            smartphone_details.append({
                'Brand Name': specs.get('Brand', '-'),
                'Smartphone Name': title,
                'Colour': specs.get('Color', '-'),
                'RAM': specs.get('RAM', '-'),
                'Storage(ROM)': specs.get('Internal Storage', '-'),
                'Primary Camera': specs.get('Primary Camera', '-'),
                'Secondary Camera': specs.get('Secondary Camera', '-'),
                'Display Size': specs.get('Display Size', '-'),
                'Battery Capacity': specs.get('Battery Capacity', '-'),
                'Price': price,
                'Product URL': url
            })
        except:
            continue
    
    driver.quit()
    
    df = pd.DataFrame(smartphone_details)
    df.to_csv(f'{search_query}_flipkart_smartphones.csv', index=False)
    return df

# Input from user
search_query = input("Enter the smartphone to search on Flipkart: ")
df = get_flipkart_smartphones(search_query)
print(df)


In [ ]:
#5
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def get_geospatial_coordinates(city_name):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://maps.google.com/')
    
    search_box = driver.find_element(By.ID, 'searchboxinput')
    search_box.send_keys(city_name)
    search_box.submit()
    
    time.sleep(5)
    
    url = driver.current_url
    driver.quit()
    
    coords = url.split('@')[1].split(',')[0:2]
    latitude, longitude = coords[0], coords[1]
    
    return latitude, longitude

# Input from user
city_name = input("Enter the city name to search on Google Maps: ")
latitude, longitude = get_geospatial_coordinates(city_name)
print(f"Latitude: {latitude}, Longitude: {longitude}")


In [ ]:
#6
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def get_gaming_laptops():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')
    
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    laptops = soup.find_all('div', 'TopNumbeHeading sticky-footer')
    
    laptop_details = []

    for laptop in laptops:
        try:
            name = laptop.find('h3').text.strip()
            specs = laptop.find_all('div', 'Spcs-details')
            specs = {s.text.split(":")[0].strip(): s.text.split(":")[1].strip() for s in specs}
            
            laptop_details.append({
                'Name': name,
                'Processor': specs.get('Processor', '-'),
                'Graphics': specs.get('Graphics', '-'),
                'RAM': specs.get('Ram Included', '-'),
                'Storage': specs.get('Storage', '-'),
                'Price': specs.get('Price', '-')
            })
        except:
            continue
    
    driver.quit()
    
    df = pd.DataFrame(laptop_details)
    df.to_csv('gaming_laptops_digit.csv', index=False)
    return df

df = get_gaming_laptops()
print(df)


In [ ]:
#7
import pandas as pd
from bs4 import BeautifulSoup
import requests

def get_forbes_billionaires():
    url = 'https://www.forbes.com/billionaires/'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    billionaires = soup.find_all('div', {'class': 'TableRow_row__L-0Km'})
    
    billionaire_details = []

    for person in billionaires:
        try:
            rank = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__leaderboardRank__l4hx-'}).text.strip()
            name = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__name__YljcU'}).text.strip()
            net_worth = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__netWorth__bNiaH'}).text.strip()
            age = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__age__Z2vvm'}).text.strip()
            citizenship = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__countryOfCitizenship__bapTF'}).text.strip()
            source = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__source__Wvd_F'}).text.strip()
            industry = person.find('div', {'class': 'Table_dataCell__mEu2b Table_dataCell__industry__Qq8az'}).text.strip()
            
            billionaire_details.append({
                'Rank': rank,
                'Name': name,
                'Net worth': net_worth,
                'Age': age,
                'Citizenship': citizenship,
                'Source': source,
                'Industry': industry
            })
        except:
            continue
    
    df = pd.DataFrame(billionaire_details)
    df.to_csv('forbes_billionaires.csv', index=False)
    return df

df = get_forbes_billionaires()
print(df)


In [ ]:
#8
import os
import pandas as pd
from googleapiclient.discovery import build

def get_youtube_comments(api_key, video_id, max_results=100):
    youtube = build('youtube', 'v3', developerKey=api_key)

    comments = []
    next_page_token = None

    while len(comments) < max_results:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            maxResults=100,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'Comment': comment['textDisplay'],
                'Upvotes': comment['likeCount'],
                'Posted At': comment['publishedAt']
            })

            # Stop if we have enough comments
            if len(comments) >= max_results:
                break

        next_page_token = response.get('nextPageToken')

        # Stop if there are no more pages
        if next_page_token is None:
            break

    return comments

def save_comments_to_csv(comments, filename='youtube_comments.csv'):
    df = pd.DataFrame(comments)
    df.to_csv(filename, index=False)

if __name__ == "__main__":
    api_key = 'YOUR_YOUTUBE_API_KEY'  # Replace with your YouTube API key
    video_id = 'VIDEO_ID'  # Replace with the YouTube video ID
    max_results = 500

    comments = get_youtube_comments(api_key, video_id, max_results)
    save_comments_to_csv(comments)

    print(f"Extracted {len(comments)} comments and saved to 'youtube_comments.csv'")


In [ ]:
#9
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def get_london_hostels():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://www.hostelworld.com/')
    
    # Enter the location and search
    search_box = driver.find_element(By.ID, 'search-input-field')
    search_box.send_keys('London')
    search_box.submit()
    
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    hostels = soup.find_all('div', 'property')
    
    hostel_details = []

    for hostel in hostels:
        try:
            name = hostel.find('h2', 'title').text.strip()
            distance = hostel.find('div', 'distance').text.strip()
            rating = hostel.find('div', 'rating-score').text.strip()
            total_reviews = hostel.find('div', 'reviews').text.strip()
            overall_reviews = hostel.find('div', 'keyword').text.strip()
            privates_from = hostel.find('div', 'price').text.strip()
            dorms_from = hostel.find('div', 'price').text.strip()
            facilities = hostel.find('div', 'facilities').text.strip()
            description = hostel.find('div', 'description').text.strip()
            
            hostel_details.append({
                'Hostel Name': name,
                'Distance from City Centre': distance,
                'Ratings': rating,
                'Total Reviews': total_reviews,
                'Overall Reviews': overall_reviews,
                'Privates From Price': privates_from,
                'Dorms From Price': dorms_from,
                'Facilities': facilities,
                'Property Description': description
            })
        except:
            continue
    
    driver.quit()
    
    df = pd.DataFrame(hostel_details)
    df.to_csv('london_hostels.csv', index=False)
    return df

df = get_london_hostels()
print(df)
